In [1]:
import pandas as pd
import numpy as np
import os 
import shutil
from tqdm import tqdm
import copy
import random

import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset , DataLoader
import torchvision.models as models 
import torchvision.models._utils as _utils


from albumentations.core.transforms_interface import ImageOnlyTransform
import albumentations as A
import albumentations.pytorch


from efficientnet_pytorch import EfficientNet
    
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import time
import cv2
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline  
%config InlineBackend.figure_format='retina'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# hyperparameter 
img_path = 'input/data/train/images'
batch_size = 64
num_workers = 2
learning_rate = 0.001
epochs = 100
img_size = 256

In [3]:
df = pd.read_csv('preprocessing_data.csv')
df.drop(columns=['Unnamed: 0'], inplace = True)
df.head()

,path,id,mask,gender,age,label
0,input/data/train/images/000001_female_Asian_45...,000001,0,1,1,4
1,input/data/train/images/000001_female_Asian_45...,000001,2,1,1,16
2,input/data/train/images/000001_female_Asian_45...,000001,0,1,1,4
3,input/data/train/images/000001_female_Asian_45...,000001,0,1,1,4
4,input/data/train/images/000001_female_Asian_45...,000001,1,1,1,10


In [4]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path , df ,transform = None):
        
        self.path = path
        self.df = df
        self.transform = transform
        
    def __getitem__(self,idx):
        image = cv2.imread(self.df['path'].iloc[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image) 
            image = augmented['image']
        # label = self.df['label'].iloc[idx]
        age =  self.df['age'].iloc[idx]
        gender =  self.df['gender'].iloc[idx]
        mask =  self.df['mask'].iloc[idx]
        
        labels = (age ,  gender , mask)
        
        return image, labels

    def __len__(self):
        return len(self.df)

In [5]:
from typing import List, Union, Tuple, Sequence, Optional
        
def get_center_crop_coords(img , crop_height: int, crop_width: int, ratio : int):
    transformed_img = img.copy()
    height = img.shape[1]
    width = img.shape[0]
    delta = random.uniform(-ratio,ratio)
    y1 = int(np.round((1 + delta) * (height - crop_height) // 2))
    y2 = y1 + crop_height
    x1 = int(np.round((1 + delta) * (height - crop_width) // 2))
    x2 = x1 + crop_width
    return transformed_img[y1:y2 , x1:x2]

class CustomRandomCrop(ImageOnlyTransform):
    def __init__(self, height, width, ratio, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.crop_height = height
        self.crop_width = width
        self.ratio = ratio
        
    def apply(self, img, **params):
        return get_center_crop_coords(img,self.crop_height, self.crop_width , self.ratio )

In [6]:
# albumentation 사용 

mean = [0.560 , 0.524 , 0.501]
std = [0.233 , 0.243 , 0.246]

train_transforms = A.Compose([
    CustomRandomCrop(224,224,0.1),
    A.Normalize(mean , std),
    A.pytorch.transforms.ToTensor()
])

val_transforms = A.Compose([
    CustomRandomCrop(224,224,0.1),
    A.Normalize(mean , std),
    A.pytorch.transforms.ToTensor()
])

In [7]:
train_x , val_x , train_y , val_y = train_test_split(df , df['label'] , test_size=0.3 , shuffle = False)

train_dataset = CustomDataset(img_path , train_x , train_transforms)
val_dataset = CustomDataset(img_path , val_x , val_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size , shuffle = True , num_workers = num_workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size , shuffle = True , num_workers = num_workers)

In [9]:
# return_layers = {'layer2': 1, 'layer3': 2, 'layer4': 3}
# https://github.com/biubug6/Pytorch_Retinaface

class Conv(nn.Module):
    def __init__(self, in_channels , out_channels , kernel_size):
        super(Conv, self).__init__()
        if kernel_size == 1 :
            self.padding = 0 
            self.stride = 2
        else : 
            self.padding = 1
            self.stride = 1
        self.out_channels = out_channels
        
        self.conv = nn.Conv2d(in_channels = in_channels , 
                            out_channels = 1024,
                            kernel_size = kernel_size,
                            stride = self.stride ,
                             padding = self.padding )
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1024, 128)
        self.ReLU = nn.ReLU(True)
        self.fc2 = nn.Linear(128 ,self.out_channels)
    
    def forward(self,x):
        x = self.conv(x)
        x = self.global_avg_pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.ReLU(x)
        x = self.fc2(x)

        return x

####################################################################################################

class AgeHead(nn.Module):
    def __init__(self, Conv):
        super(AgeHead,self).__init__()
        self.conv = Conv(in_channels = 2048 , 
                            out_channels = 3,
                            kernel_size = 3)
    def forward(self,x):
        x = self.conv(x)
        return x

class GenderHead(nn.Module):
    def __init__(self, Conv):
        super(GenderHead,self).__init__()
        self.conv = Conv(in_channels = 2048 , 
                            out_channels = 2,
                            kernel_size = 3)

    def forward(self,x):
        x = self.conv(x)
        return x
    
class MaskHead(nn.Module):
    def __init__(self, Conv):
        super(MaskHead,self).__init__()
        self.conv = Conv(in_channels = 2048 , 
                            out_channels = 3,
                            kernel_size = 3)

    def forward(self,x):
        x = self.conv(x)
        return x

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel , self).__init__()
        backbone = models.resnet50(pretrained = True)
        self.backbone = torch.nn.Sequential(*(list(backbone.children())[:-2])) # (-1, 2048,8,8)
        self.AgeHead = AgeHead(Conv)
        self.GenderHead = GenderHead(Conv)
        self.MaskHead = MaskHead(Conv)
        
    def forward(self, x):
        x = self.backbone(x)
        y1 = self.AgeHead(x)
        y2 = self.GenderHead(x)
        y3 = self.MaskHead(x)
        out = (y1 , y2 ,y3)
        return out

In [10]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None , alpha = 0.25,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob * self.alpha,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

class MultiLoss(nn.Module):
    """
    ground_truth(tuple) : Ground truth labels for a batch
        shape : [batch_size , 3] -> 3 = (age , gender , mask)
    prediction(tuple) : 
        shape : [batch_size , num_classes , 3]
    """
    
    def __init__(self, gamma = 2 , alpha = 0.25 , lamb = 1, size_average = True ):
        super(MultiLoss , self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.loss = FocalLoss(alpha = self.alpha , gamma = self.gamma)
        self.lamb = lamb
        
    def forward(self, y_pred ,target):
        
        pred_age , pred_gender , pred_mask = y_pred     # (batch_size , 3) , (batch_size , 2) , (batch_size , 3)
        target_age , target_gender , target_mask = target
        
        age_loss = self.loss(pred_age , target_age).to(device)
        gender_loss = self.loss(pred_gender , target_gender).to(device)
        mask_loss = self.loss(pred_mask , target_mask).to(device)
     
        total_loss =  gender_loss + mask_loss + self.lamb * age_loss
        return total_loss

In [11]:
def train(epochs , train_loader, val_loader , model , criterion , optimizer , lr_scheduler , num_early_stop = 5):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch = 0
    best_val_loss = 999999999
    eps  = 1e-8
    early_stop = 0
    
    for epoch in range(epochs):
        
        if early_stop >= num_early_stop : break
            
        ##################################### train ################################
        model.train()
        
        loss_train_sum = 0
        acc_train_sum = 0
        f1_train_sum = 0
        
        for i , (img , target) in enumerate(tqdm(train_loader)):
            img = img.to(device)
            target = [t.to(device) for t in target]

            y_pred = model.forward(img)
            loss = criterion(y_pred, target)
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train_sum += loss
            
            acc_train = 0 
            f1_train = 0
            
            ## Cal acc, macrof1
            for i in range(3):
                pred = y_pred[i].argmax(1)
                tgt = target[i]
                acc_train += (pred == tgt).sum().item()
                f1_train += f1_score(tgt.data.detach().cpu(), pred.detach().cpu(), average='macro')
                
            acc_train_sum += (acc_train / 3)
            f1_train_sum += (f1_train / 3)
        
        loss_train_avg = loss_train_sum / len(train_loader)
        acc_train_avg = acc_train_sum / len(train_loader)
        f1_train_avg = f1_train_sum /  len(train_loader)
        print(f" epoch:[{epoch+1}/{epochs}] cost:[{loss_train_avg:.3f}] acc : [{acc_train_avg : .3f}]  f1: [{f1_train_avg : .3f}]" )
        print(f" learning rate : {lr_scheduler.get_last_lr()}")
        
        ##################################### eval ################################
        model.eval()
        
        loss_val_sum = 0
        acc_val_sum = 0
        f1_val_sum = 0
        
        for i , (img , target) in enumerate(tqdm(val_loader)):
            img = img.to(device)
            target = [t.to(device) for t in target]
            
            with torch.no_grad():
                y_pred = model.forward(img)
                loss = criterion(y_pred, target)
            
            loss_val_sum += loss
            
            acc_val = 0 
            f1_val = 0
            
            ## Cal acc, macrof1
            for i in range(3):
                pred = y_pred[i].argmax(1)
                tgt = target[i]
                acc_val += (pred == tgt).sum().item()
                f1_val += f1_score(tgt.data.detach().cpu(), pred.detach().cpu(), average='macro')
                
            acc_val_sum += (acc_val / 3)
            f1_val_sum += (f1_val / 3)
                    
        
        loss_val_avg = loss_val_sum / len(val_loader)
        acc_val_avg = acc_val_sum / len(val_loader)
        f1_val_avg = f1_val_sum /  len(val_loader)
        
        print(f" epoch:[{epoch+1}/{epochs}] cost:[{loss_val_avg:.3f}] acc : [{acc_val_avg : .3f}] f1: [{f1_val_avg : .3f}]")
        
        if lr_scheduler : 
            lr_scheduler.step()
        
 
        if best_val_loss > loss_val_avg:
            best_val_loss = loss_val_avg
            best_epoch = epoch + 1
            best_model_wts = copy.deepcopy(model.state_dict())
            
            early_stop = 0
        
        else :
            early_stop += 1
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), f'NewModel_add_fc_base_{best_epoch}.pt')

In [12]:
Model = BaseModel().to(device)
criterion =  MultiLoss(gamma = 2 , alpha = 0.25 , lamb = 2)
optimizer = optim.AdamW([
        {'params': Model.backbone.parameters(), 'lr': 3e-4}
    ] , lr = learning_rate , weight_decay=0.005)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 5, T_mult = 2, eta_min=1e-6)

In [13]:
train(epochs , train_loader , val_loader , Model, criterion , optimizer , lr_scheduler)

  0%|          | 0/89 [00:00<?, ?it/s]

 epoch:[1/100] cost:[0.052] acc : [ 60.432]  f1: [ 0.909]
 learning rate : [0.0003]


  0%|          | 0/207 [00:00<?, ?it/s]

 epoch:[1/100] cost:[0.092] acc : [ 58.120] f1: [ 0.822]


  0%|          | 0/89 [00:00<?, ?it/s]

...

 epoch:[8/100] cost:[0.109] acc : [ 59.127] f1: [ 0.848]


  0%|          | 0/89 [00:00<?, ?it/s]

 epoch:[9/100] cost:[0.003] acc : [ 63.651]  f1: [ 0.993]
 learning rate : [0.0002383738952177247]


100%|██████████| 89/89 [00:14<00:00,  6.25it/s]


 epoch:[9/100] cost:[0.127] acc : [ 59.809] f1: [ 0.856]


In [38]:
Model = BaseModel().to(device)
Model.load_state_dict(torch.load('NewModel_add_fc_base_4.pt'))

<All keys matched successfully>

In [39]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [34]:
test_dir = 'input/data/eval'

In [40]:
def Cal_label(pred_age , pred_gender , pred_mask):
    """
    pred_age : [batch_size , 3]
    pred_gender : [batch_size , 2]
    pred_mask : [batch_size , 3]
    """
    age = pred_age.argmax(dim=-1)
    gender = pred_gender.argmax(dim=-1)
    mask = pred_mask.argmax(dim=-1)
    
    multi_class_label = mask * 6 + gender * 3 + age
    return multi_class_label

In [41]:
img_size = 224
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
                                    transforms.Resize((img_size, img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
Model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred_age , pred_gender , pred_mask = Model(images)
        pred = Cal_label(pred_age , pred_gender , pred_mask)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'New_model_submission.csv'), index=False)
print('test inference is done!')

test inference is done!


In [1]:
import timm
model = timm.create_model('efficientnet_b3' , pretrained = True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


In [3]:
from torchsummary import summary
summary(model.cuda(), (3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 40, 128, 128]           1,080
       BatchNorm2d-2         [-1, 40, 128, 128]              80
           SwishMe-3         [-1, 40, 128, 128]               0
            Conv2d-4         [-1, 40, 128, 128]             360
       BatchNorm2d-5         [-1, 40, 128, 128]              80
           SwishMe-6         [-1, 40, 128, 128]               0
            Conv2d-7             [-1, 10, 1, 1]             410
           SwishMe-8             [-1, 10, 1, 1]               0
            Conv2d-9             [-1, 40, 1, 1]             440
    SqueezeExcite-10         [-1, 40, 128, 128]               0
           Conv2d-11         [-1, 24, 128, 128]             960
      BatchNorm2d-12         [-1, 24, 128, 128]              48
         Identity-13         [-1, 24, 128, 128]               0
DepthwiseSeparableConv-14         [-1, 

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
import timm
from timm.models.layers.classifier import ClassifierHead
import torch.nn.utils.weight_norm as weightNorm

class MultiBranchModel(nn.Module):
    
    """
    MultiBranchModel 
        - output (tuppe):([batch_size , 3] ,[batch_size , 2] , [batch_size , 3] )
    """
    
    def __init__(self, num_classes, dropout_num = 4 ,dropout_p = 0.5):
        super().__init__()
        model = timm.create_model('efficientnet_b0' , pretrained = True)
        self.backbone = torch.nn.Sequential(*(list(model.children())[:-1]))
        self.dropout_num = dropout_num
        self.maskdropouts = nn.ModuleList([nn.Dropout(dropout_p) for _ in range(dropout_num)])
        self.maskfc = nn.Linear(1280 , num_classes)
        
        nn.init.normal_(self.maskfc.weight, std=0.001)
        nn.init.constant_(self.maskfc.bias, 0) 
        
        self.genderdropouts = nn.ModuleList([nn.Dropout(dropout_p) for _ in range(dropout_num)])
        self.genderfc = nn.Linear(1280 , num_classes-1)
        nn.init.normal_(self.genderfc.weight, std=0.001)
        nn.init.constant_(self.genderfc.bias, 0)   
        
        self.agedropouts = nn.ModuleList([nn.Dropout(dropout_p) for _ in range(dropout_num)])
        self.agefc = nn.Linear(1280 , num_classes)
        nn.init.normal_(self.agefc.weight, std=0.001)
        nn.init.constant_(self.agefc.bias, 0)
        

    def forward(self, x):
        x = self.backbone(x)
        # Mask
        for i,dropout in enumerate(self.maskdropouts):
            if i== 0:
                mask = self.maskfc(dropout(x))
            else : 
                mask += self.maskfc(dropout(x)) 
        mask /= self.dropout_num
        
        # Gender
        for i,dropout in enumerate(self.genderdropouts):
            if i== 0:
                gender = self.genderfc(dropout(x))
            else : 
                gender += self.genderfc(dropout(x))         
        gender /= self.dropout_num
        
        # Age
        for i,dropout in enumerate(self.agedropouts):
            if i== 0:
                age = self.agefc(dropout(x))
            else : 
                age += self.agefc(dropout(x))
        age /= self.dropout_num
        
        return (mask, gender, age) 


In [47]:
model = MultiBranchModel(3)
for name, param in model.named_parameters():
    if name in [ 'maskfc.weight' , 'maskfc.bias' ,'genderfc.weight' ,'genderfc.bias']:
        param.requires_grad = False

In [48]:
for name, param in model.named_parameters():
    if not param.requires_grad :
        print(name)

maskfc.weight
maskfc.bias
genderfc.weight
genderfc.bias


In [42]:
for param in model.parameters():
    if param in [ 'maskfc.weight' , 'maskfc.bias' ,'genderfc.weight' ,'genderfc']:
        param.requires_grad = False

In [43]:
for param in model.parameters():
    if param.param.requires_grad == False :
        print(param)

AttributeError: 'Parameter' object has no attribute 'param'

In [50]:
model = timm.create_model('efficientnet_b3' , pretrained = True)